<a href="https://colab.research.google.com/github/mraskj/css_fall2023/blob/main/code/class10/class10-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class 10: Audio Measurement - Tutorial

In this tutorial, we will explore how Python enables you to interact with Praat in a Pythonic way. Praat is a popular speech analysis software designed to study phonetics (the sound of speeches) using computers. It offers a way of functionalities such as speech synthesis (i.e. text-to-speech), speech manipulation, and also machine learning algorithms such as PCA. In this course, we will focus on its speech analysis capabilities, which is considered the best possible option when one wants to computationally analyze speeches using lingustic and phonetic theory.

Below I have printed the same list of useful Colab shortcuts as in *class09*.

- Add a text cell: `ctrl + L`
- Add a code cell: `ctrl + K`
- Convert to text cell: `ctrl + M M`
- Convert to code cell: `ctrl + M Y`
- Add code cell above: `ctrl + M A`
- Add code cell below: `ctrl + M B`
- Run cell and select new cell `shift + enter`
- Run cell and insert new cell `alt + enter`
- Run selected code `ctrl + shift + enter`
- Run the focused cell `ctrl + enter`
- Run all cells in notebook `ctrl + F9`
- Run all cells before the current `ctrl + F8`
- Run cell and all cells after `ctrl + F10`
- Restart runtime: `ctrl + M`


## 0 Setup

### 0.1 Accessing GitHub
To access the audio files from GitHub, we will start by cloning the GitHub repo into our local runtime. This can be with the terminal command:

```
!git clone https://github.com/mraskj/css_fall2023.git
```

The exclamation mark `!` specifies that we execute a terminal command within a notebook, just as we have seen previously. When the repo is cloned, it is located in */content/

In [ ]:
# Clone GitHub directory into
!git clone https://github.com/mraskj/css_fall2023.git


### 0.2 Environments and Packages

Unlike local notebooks, which we typically run in local environments to keep package dependencies in check, Colab notebooks run each runtime. That means that all packages must be installed if you for instance refresh Colab. Unlike local environments, however, Colab has a bunch of preinstalled packages, but we still need to a few on certain occassions. We can do that using


```
!pip install -r /content/css_fall2023/requirements/FILENAME.txt
```

where *FILENAME* is the name of the file (e.g. *requirements-topic4-class9*).

In [ ]:
!pip install -r /content/css_fall2023/requirements/requirements_topic4-class9.txt

### 0.3 Importing Modules

In [ ]:
# Importing modules

# For file and directory management
import os

# For regular expressions
import re

# For data handling
import numpy as np
import pandas as pd

# For plotting
import seaborn as sns
import matplotlib.pyplot as plt
sns.set() # Use seaborn's default style to make attractive graphs

# Feature extraction
import librosa

# For Praat
import parselmouth

# For math functionalities
import math

### 0.4 Reading Data

For the lab session for *class10*, we work with 20 audio files each corresponding to a speech in the Danish parliament in the second assembly of the 2007 parliamentary term (*20072*). The files come from 10 different speakers with two speeches each. FIve of the speakers are men and Five are women.

You find the files in *data/audio/class10/* folder in the cloned GitHub repo.

In [ ]:
# Define path to directory containing audio files - os.getcwd() corresponds to '/content'
base_dir = os.path.join(os.getcwd(), 'css_fall2023/data/audio/class10')

# List files in the `male` and `female` folder
male_files = os.listdir(os.path.join(base_dir, 'male'))
female_files = os.listdir(os.path.join(base_dir, 'female'))

If we print `male_files` and `female_files`, we'll see that they are unsorted. To keep things in order, we sort the files by the first digits in the filenames. We define a custom function for this purpose and provide the function as parameter to the `key` argument in the `sorted()` function.

In [ ]:
# Define a custom sorting key function
def digit_sort_key(v):

    digits = re.search('\d+', v)
    return int(digits.group())
    #if digits:
    #    return int(digits.group())
    #else:
    #    return float('inf')  # Return infinity for strings with no numbers

# Sort the lists
male_files = sorted(male_files, key=digit_sort_key)
female_files = sorted(female_files, key=digit_sort_key)

# Print to see that the lists are sorted
female_files, male_files

## 1 Praat Sound Objects

In *class09*, we loaded our audio using `wavfile.read()` method from the `scipy.io` module. When we use `parselmouth-praat`, we need to explicitly tell Python that our audio file is supposed to interact with *Praat*. We do that by constructing a `parselmouth.Sound` object.


In [ ]:
# Initiate sound object
snd = parselmouth.Sound(os.path.join(base_dir, 'female', female_files[0]))

print(type(snd))

In [ ]:
print(snd)

In [ ]:
# To see all properties and methods of the object
dir(snd)

In [ ]:
#snd.sampling_frequency, snd.sampling_period
#snd.get_number_of_samples(), snd.get_number_of_channels()
#snd.xmin, snd.xmax, snd.get_root_mean_square, snd.get_rms
#snd.get_total_duration()
#snd.get_energy_in_air(), snd.get_energy()
#snd.get_intensity()
#snd.get_maximum(), snd.get_minimum()

In [ ]:
# Access time samples
t = snd.xs()
print(f"Number of samples: {t.shape}")
print(f"Last time sample: {t[-1]}")
print(type(t))

In [ ]:
# Access values
amp = snd.values
print(f"Number of samples: {amp.shape}")
print(type(amp))

We are now back in pure Python language as `t` and `amp` are both of type `numpy.ndarray`

To illustrate its content, we plot a snippet of the first audio file in the `female_list` as a waveform. For the purpose, we define a function.

In [ ]:
# Define function to plot a waveform
def draw_waveform(x, y,
                  color='#381a61',
                  alpha=.5,
                  xlim=[0.0, 1.0],
                  xlabel=None,
                  ylabel=None,
                  show=True):

  plt.figure(figsize=(7,5))
  plt.plot(x, y, alpha=alpha, color=color)
  plt.xlim(xlim)

  if xlabel:
    plt.xlabel(xlabel)

  if ylabel:
    plt.ylabel(ylabel)

  if show:
    plt.show()

In [ ]:
# Plot the first 16000 samples = 1 second of audio
sr = snd.sampling_frequency
start, stop = 0, 16000
x, y = snd.xs()[start:stop], snd.values[0][start:stop].T
draw_waveform(x,
              y,
              xlim=[start, stop/sr],
              xlabel='Time (s)', ylabel='Amplitude')

## 2 Feature Extraction with Praat

The `parselmouth-praat` makes it straightforward to construct audio measures from the `parselmouth.Sound` object.

This can be done with two approaches:
* `to_()`
* `call()`


In [ ]:
# to_()
mfccs = snd.to_mfcc(10)
pitch = snd.to_pitch()
harmonicity = snd.to_harmonicity()
intensity = snd.to_intensity()
spectrogram = snd.to_spectrogram()
spectrum = snd.to_spectrum()

In [ ]:
# call()
f0min, f0max = 100, 500
pitch_obj = parselmouth.praat.call(snd, "To Pitch", 0.0, f0min, f0max)
meanF0 = parselmouth.praat.call(pitch_obj, "Get mean", 0, 0, 'Hertz')
stdevF0 = parselmouth.praat.call(pitch_obj, "Get standard deviation", 0 ,0, 'Hertz')
maxF0 = parselmouth.praat.call(pitch_obj, "Get maximum", 0, 0, 'Hertz',"None")
minF0 = parselmouth.praat.call(pitch_obj, "Get minimum", 0, 0, 'Hertz',"None")

In [ ]:
# Access time samples and pitch estimates from the `pitch` object
# pitch.xs()
# pitch.values

In [ ]:
# Use custom settings
# pitch = snd.to_pitch(pitch_floor=100, pitch_ceiling=500)

## 3) Plotting

In [ ]:
def draw_spectrogram(spectrogram, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='viridis')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

def draw_intensity(intensity):
    plt.plot(intensity.xs(), intensity.values.T, linewidth=3, color='w')
    plt.plot(intensity.xs(), intensity.values.T, linewidth=1)
    plt.grid(False)
    plt.ylim(0)
    plt.ylabel("intensity [dB]")

def draw_pitch(pitch):
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")

In [ ]:
# Plot spectrogram and intensity + spectrogram and F0
start, stop = 3, 4
pitch = snd.to_pitch()
spectrogram = snd.to_spectrogram()
intensity = snd.to_intensity()

plt.figure(figsize=(24, 12))
plt.subplot(1, 2, 1)
draw_spectrogram(spectrogram)
plt.twinx()
draw_intensity(intensity)
plt.xlim([start, stop])

plt.subplot(1, 2, 2)
draw_spectrogram(spectrogram)
plt.yticks([])
plt.twinx()
draw_pitch(pitch)
plt.xlim([start, stop])

plt.show()

In [ ]:
# Function to compute the speech rate of audio files
# All credit to Dr. Feinberg (https://github.com/drfeinberg/PraatScripts/blob/master/syllable_nuclei.py)
def speech_rate(file):

    silencedb = -25
    mindip = 2
    minpause = 0.3
    sound = parselmouth.Sound(file)
    originaldur = sound.get_total_duration()
    intensity = sound.to_intensity(50)
    start = parselmouth.praat.call(intensity, "Get time from frame number", 1)
    nframes = parselmouth.praat.call(intensity, "Get number of frames")
    end = parselmouth.praat.call(intensity, "Get time from frame number", nframes)
    min_intensity = parselmouth.praat.call(intensity, "Get minimum", 0, 0, "Parabolic")
    max_intensity = parselmouth.praat.call(intensity, "Get maximum", 0, 0, "Parabolic")

    # get .99 quantile to get maximum (without influence of non-speech sound bursts)
    max_99_intensity = parselmouth.praat.call(intensity, "Get quantile", 0, 0, 0.99)

    # estimate Intensity threshold
    threshold = max_99_intensity + silencedb
    threshold2 = max_intensity - max_99_intensity
    threshold3 = silencedb - threshold2
    if threshold < min_intensity:
        threshold = min_intensity

    # get pauses (silences) and speakingtime
    textgrid = parselmouth.praat.call(intensity, "To TextGrid (silences)", threshold3, minpause, 0.1, "silent", "sounding")
    silencetier = parselmouth.praat.call(textgrid, "Extract tier", 1)
    silencetable = parselmouth.praat.call(silencetier, "Down to TableOfReal", "sounding")
    npauses = parselmouth.praat.call(silencetable, "Get number of rows")
    speakingtot = 0
    for ipause in range(npauses):
        pause = ipause + 1
        beginsound = parselmouth.praat.call(silencetable, "Get value", pause, 1)
        endsound = parselmouth.praat.call(silencetable, "Get value", pause, 2)
        speakingdur = endsound - beginsound
        speakingtot += speakingdur

    intensity_matrix = parselmouth.praat.call(intensity, "Down to Matrix")
    # sndintid = sound_from_intensity_matrix
    sound_from_intensity_matrix = parselmouth.praat.call(intensity_matrix, "To Sound (slice)", 1)
    # use total duration, not end time, to find out duration of intdur (intensity_duration)
    # in order to allow nonzero starting times.
    intensity_duration = parselmouth.praat.call(sound_from_intensity_matrix, "Get total duration")
    intensity_max = parselmouth.praat.call(sound_from_intensity_matrix, "Get maximum", 0, 0, "Parabolic")
    point_process = parselmouth.praat.call(sound_from_intensity_matrix, "To PointProcess (extrema)", "Left", "yes", "no", "Sinc70")
    # estimate peak positions (all peaks)
    numpeaks = parselmouth.praat.call(point_process, "Get number of points")
    t = [parselmouth.praat.call(point_process, "Get time from index", i + 1) for i in range(numpeaks)]

    # fill array with intensity values
    timepeaks = []
    peakcount = 0
    intensities = []
    for i in range(numpeaks):
        value = parselmouth.praat.call(sound_from_intensity_matrix, "Get value at time", t[i], "Cubic")
        if value > threshold:
            peakcount += 1
            intensities.append(value)
            timepeaks.append(t[i])

    # fill array with valid peaks: only intensity values if preceding
    # dip in intensity is greater than mindip
    validpeakcount = 0
    currenttime = timepeaks[0]
    currentint = intensities[0]
    validtime = []

    for p in range(peakcount - 1):
        following = p + 1
        followingtime = timepeaks[p + 1]
        dip = parselmouth.praat.call(intensity, "Get minimum", currenttime, timepeaks[p + 1], "None")
        diffint = abs(currentint - dip)
        if diffint > mindip:
            validpeakcount += 1
            validtime.append(timepeaks[p])
        currenttime = timepeaks[following]
        currentint = parselmouth.praat.call(intensity, "Get value at time", timepeaks[following], "Cubic")

    # Look for only voiced parts
    pitch = sound.to_pitch_ac(0.02, 30, 4, False, 0.03, 0.25, 0.01, 0.35, 0.25, 450)
    voicedcount = 0
    voicedpeak = []

    for time in range(validpeakcount):
        querytime = validtime[time]
        whichinterval = parselmouth.praat.call(textgrid, "Get interval at time", 1, querytime)
        whichlabel = parselmouth.praat.call(textgrid, "Get label of interval", 1, whichinterval)
        value = pitch.get_value_at_time(querytime)
        if not math.isnan(value):
            if whichlabel == "sounding":
                voicedcount += 1
                voicedpeak.append(validtime[time])

    # calculate time correction due to shift in time for Sound object versus
    # intensity object
    timecorrection = originaldur / intensity_duration

    # Insert voiced peaks in TextGrid
    parselmouth.praat.call(textgrid, "Insert point tier", 1, "syllables")
    for i in range(len(voicedpeak)):
        position = (voicedpeak[i] * timecorrection)
        parselmouth.praat.call(textgrid, "Insert point", 1, position, "")

    # return results
    speakingrate = voicedcount / originaldur
    articulationrate = voicedcount / speakingtot
    npause = npauses - 1
    asd = speakingtot / voicedcount
    speechrate_dictionary = {'soundname': file.split('/')[-1],
                             'nsyll':voicedcount,
                             'npause': npause,
                             'dur(s)':originaldur,
                             'phonationtime(s)':intensity_duration,
                             'speechrate(nsyll / dur)': speakingrate,
                             "articulation rate(nsyll / phonationtime)":articulationrate,
                             "ASD(speakingtime / nsyll)":asd}
    return speechrate_dictionary


In [ ]:
# Compute speech rate
speech_rate_list = []
for f in female_files:
    speechrate_dictionary = speech_rate(file = os.path.join(base_dir, 'female', f))
    speech_rate_list.append(speechrate_dictionary)

for f in male_files:
    speechrate_dictionary = speech_rate(file = os.path.join(base_dir, 'male', f))
    speech_rate_list.append(speechrate_dictionary)

speech_rate_df = pd.DataFrame(speech_rate_list)

speech_rate_df